In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime

In [ ]:
train = pd.read_csv('data/train_dropped_290_without_nans.csv')
test = pd.read_csv('data/test_dropped_290_without_nans.csv')
y_train=pd.read_csv('data/y_train.csv')['Culture']
print(train.shape, test.shape)

(2838, 293) (939, 293)


In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
xgb = XGBClassifier(learning_rate=0.01, n_estimators=800, nthread=-1)

In [ ]:
%%time
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 0)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_weighted', n_jobs=-1, cv=skf.split(train,y_train), verbose=3, random_state=42 )

# Here we go # timing starts from this point for "start_time" variable
random_search.fit(train,y_train)

c:\users\adm\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed: 12.1min remaining: 18.2min
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed: 16.5min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 17.3min finished


Wall time: 20min 32s


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000001DDDB685EC8>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.01,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, mis...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None, tr

In [ ]:
preds = random_search.predict(test)
pd.Series(preds).to_csv('preds/submission-random-grid-search-xgb-01.csv', index=False, header=['Culture'])

In [ ]:
train.drop(['0','1','2'],axis=1,inplace=True)
test.drop(['0','1','2'],axis=1,inplace=True)

In [ ]:
%%time
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_weighted', n_jobs=-1, cv=skf.split(train,y_train), verbose=3, random_state=42 )

# Here we go # timing starts from this point for "start_time" variable
random_search.fit(train,y_train)
preds = random_search.predict(test)
pd.Series(preds).to_csv('preds/submission-random-grid-search-xgb-290.csv', index=False, header=['Culture'])

c:\users\adm\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed: 32.5min remaining: 10.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 35.0min finished


Wall time: 36min 34s


In [ ]:
random_search.best_params_

{'subsample': 0.8,
 'min_child_weight': 10,
 'max_depth': 4,
 'gamma': 1.5,
 'colsample_bytree': 0.6}

In [ ]:
%%time
params = {
        'min_child_weight': [10],
        'gamma': [1.5],
        'subsample': [0.8],
        'colsample_bytree': [0.6],
        'max_depth': [6]
        }
xgb = XGBClassifier(learning_rate=0.01, n_estimators=1200, nthread=-1)
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 256)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_weighted', n_jobs=-1, cv=skf.split(train,y_train), verbose=3, random_state=42 )

# Here we go # timing starts from this point for "start_time" variable
random_search.fit(train,y_train)
preds = random_search.predict(test)
pd.Series(preds).to_csv('preds/submission-random-grid-search-xgb-290.csv', index=False, header=['Culture'])

c:\users\adm\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
c:\users\adm\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:282: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  9.0min remaining: 13.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  9.1min finished


Wall time: 11min 26s


In [3]:
train = pd.read_csv('train_290_without_low1.csv')
test = pd.read_csv('test_dropped_290_without_nans.csv')
print(train.shape, test.shape)

(2828, 294) (939, 293)


In [4]:
%%time
params = {
        'min_child_weight': [10],
        'gamma': [1.5],
        'subsample': [0.8],
        'colsample_bytree': [0.6],
        'max_depth': [6]
        }
xgb = XGBClassifier(learning_rate=0.01, n_estimators=1200, nthread=-1)
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 256)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_weighted', n_jobs=-1, cv=skf.split(train.drop('Culture',axis=1),train['Culture']), verbose=3, random_state=42 )

# Here we go # timing starts from this point for "start_time" variable
random_search.fit(train.drop('Culture',axis=1),train['Culture'])
preds = random_search.predict(test)
pd.Series(preds).to_csv('xgb_51.csv', index=False, header=['Culture'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 29.4min finished


CPU times: user 15min 22s, sys: 1.49 s, total: 15min 24s
Wall time: 37min 14s


In [5]:
train = pd.read_csv('train_290_without_low2.csv')
test = pd.read_csv('test_dropped_290_without_nans.csv')
print(train.shape, test.shape)

(2794, 294) (939, 293)


In [6]:
%%time
params = {
        'min_child_weight': [10],
        'gamma': [1.5],
        'subsample': [0.8],
        'colsample_bytree': [0.6],
        'max_depth': [6]
        }
xgb = XGBClassifier(learning_rate=0.01, n_estimators=1200, nthread=-1)
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 256)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_weighted', n_jobs=-1, cv=skf.split(train.drop('Culture',axis=1),train['Culture']), verbose=3, random_state=42 )

# Here we go # timing starts from this point for "start_time" variable
random_search.fit(train.drop('Culture',axis=1),train['Culture'])
preds = random_search.predict(test)
pd.Series(preds).to_csv('xgb_52.csv', index=False, header=['Culture'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 25.8min finished


CPU times: user 13min 34s, sys: 1.31 s, total: 13min 36s
Wall time: 32min 43s
